In [88]:
from ultralytics import YOLO
import cv2
import numpy as np

In [106]:
model_path = "D:/Betafinder/Betafinder/models/object_detection/yolo_weights/bounding_and_color.pt"
img_path = "D:/Betafinder/data/Object_detection_holds/IMG20240218163706.jpg"

In [107]:
# initialize model and predict
model = YOLO(model_path)
results = model(img_path, save = True, save_txt = True)


image 1/1 D:\Betafinder\data\Object_detection_holds\IMG20240218163706.jpg: 640x480 7 Blacks, 1 Gray, 6 Pinks, 3 UNKNOWNs, 88.8ms
Speed: 3.0ms preprocess, 88.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)
Results saved to runs\detect\predict7
1 label saved to runs\detect\predict7\labels


: 

In [91]:
### Color classes identified:
print(results[0].names)

{0: 'Black', 1: 'Blue', 2: 'Gray', 3: 'Green', 4: 'Orange', 5: 'Pink', 6: 'Purple', 7: 'Red', 8: 'UNKNOWN', 9: 'White', 10: 'Yellow'}


In [92]:
### Converts the bounding box metric xywh from Yolo output to x1, x2, y1, y2 for chaining
### with cv2. Keeps the predicted color and color name of each hold as well.
def yolobbox2bbox(out):
    all_boxes = []
    plc = out.boxes.xywh
    clr = out.boxes.cls
    clr_names = out.names

    def center_border_to_xxyy(x,y,w,h):
        x1, y1 = x-w//2, y-h//2
        x2, y2 = x+w//2, y+h//2
        return x1, y1, x2, y2

    for i in range(len(plc)):
        to_int = [int(x) for x in plc[i]]
        color = int(clr[i])
        clr_name = clr_names[color]
        all_boxes.append(((color, clr_name), center_border_to_xxyy(*to_int)))

    return all_boxes

In [96]:
# Sanity-check
all_boxes = yolobbox2bbox(results[0])
pic = cv2.imread(img_path)

# Unpack all_boxes as two tuples:
(c, c_name), (x,y,w,h) = all_boxes[np.random.randint(0, len(all_boxes)-1)]

crop_img = pic[y:h, x:w]
print(c, c_name)
cv2.imshow("test", crop_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

5 Pink
